# Plate Mapper

## Instructions

1. run first cell and selected inputs `csv` and output directory
2. Enter something or nother and do the things
3. ...
4. save plate, meta, random csvs

In [1]:
import platemapper
import pandas as pd
import re # REEEEE

from IPython.display import *
import ipywidgets as widgets
from ipywidgets import *
from ipyfilechooser import FileChooser

import seaborn as sns
import matplotlib as plt


sfc = FileChooser()
sfc.title = 'Select source csv'
fc = platemapper.make_chooser('output directory')
display(sfc)
display(fc)

FileChooser(path='/hdd/project/plate_generator', filename='', title='HTML(value='Select source csv')', show_hi…

FileChooser(path='/hdd/project/plate_generator', filename='', title='HTML(value='output directory')', show_hid…

In [10]:
source = sfc.selected
out = fc.selected_path
sp = pd.read_csv(source, encoding='utf-8')
sp.head()

,CatalogNumber,ProductName,PlateLocation,384PlateName,M.w.,CASNumber,Target,DMSO(mg/mL)MaxSolubility,DMSO(mM)MaxSolubility,water(mg/mL)MaxSolubility,...,Pathway,Formula,Form,Synonyms,SMILES,ALogP,HBA_Count,HBD_Count,RotatableBond,MW REID
0,S1002,ABT-737,A3,L1700-384-01,813.43,852808-04-9,"Bcl-2,Autophagy",100,122.9362084,<1,...,Apoptosis,C42H45ClN6O5S2,free base,NaN,CN(C)CC[C@H](CSC1=CC=CC=C1)NC1=CC=C(C=C1[N+]([...,8.198,3.0,2.0,16.0,813.43000
1,S1003,Linifanib (ABT-869),C3,L1700-384-01,375.41,796967-16-3,"CSF-1R,PDGFR,VEGFR",75,199.7815721,<1,...,Protein Tyrosine Kinase,C21H18FN5O,free base,"AL39324,RG3635",CC1=CC(NC(=O)NC2=CC=C(C=C2)C2=C3C(N)=NNC3=CC=C...,4.169,2.0,4.0,3.0,375.41000
2,S1004,Veliparib (ABT-888),E3,L1700-384-01,244.29,912444-00-9,PARP,17,69.58942241,<1,...,DNA Damage,C13H16N4O,free base,NSC 737664,"C[C@@]1(CCCN1)C1=NC2=C(C=CC=C2N1)C(N)=O |r,c:1...",0.986,2.0,3.0,2.0,244.29000
3,S1005,Axitinib,G3,L1700-384-01,386.47,319460-85-0,"c-Kit,PDGFR,VEGFR",35,90.56330375,<1,...,Protein Tyrosine Kinase,C22H18N4OS,free base,AG 013736,CNC(=O)C1=CC=CC=C1SC1=CC2=C(C=C1)C(\C=C\C1=CC=...,4.492,3.0,2.0,5.0,386.47000
4,S1006,Saracatinib (AZD0530),I3,L1700-384-01,542.03,379231-04-6,Src,35,64.57207166,<1,...,Angiogenesis,C27H32ClN5O5,free base,NaN,CN1CCN(CCOC2=CC(OC3CCOCC3)=C3C(NC4=C(Cl)C=CC5=...,3.814,7.0,1.0,8.0,542.03000


In [20]:
def make_compound(destplate, max_back, dilute, sId, sW, comp, startCol, startRow,  steps=5):
    """
    Function for button to make a compound
    args:
        destplate - str destination plate ID
        max_back - float max backfill vol
        dilute - str dilution step 1/2, log2 etc. NOTE: only 1/2 is implemented
        sID - str source plate ID
        sW - str source well ID
        comp - str compound name
        (startCol, startRow) - tuple of first col and row of compound col:[A-P][0-15], row:[2-24]
        steps - int number of dilution steps
    retruns:
        dc - dataframe of compound
    """
    transVol, backVol = platemapper.cal_transVol(max_back, dilute, steps)
    destWell = platemapper.get_wellIDs(startCol, steps, startRow)
    dc = platemapper.generate_df(sourceId=sId, sourceWell=sW, destID=destplate, compound=comp, steps=steps, 
                                 destWell=destWell, transVol=transVol, backVol=backVol)
    return dc

def button_handler(sender):
    """
    Handler function for submit button
    pulls values from the 'ui' and calls make_compound 
    then loads the dataframe into the compounds list
    """
    # values from sourceplate
    df = sp.loc[sp['ProductName'] == nameDrop.value]
    sId, sW, comp, = df['384PlateName'].values[0],df['PlateLocation'].values[0],df['ProductName'].values[0]
    
    # user input values
    dilute = diluteDrop.value
    destplate = destField.value
    max_back = int(float(fillField.value))
    dilute = diluteDrop.value
        
    # add that bad boi
    compounds.append(make_compound(destplate, max_back, dilute, sId, sW, comp, colBound.value, rowBound.value))

In [21]:

# source Compound
nameDrop = Dropdown(description='Compound',options=sp['ProductName'], value=None)

# dilution step
diluteDrop = Dropdown(description='Select Dilute amount',options=['log2', '1/2'], value=None)

# destination plate id
destField = Text(value='Destination Plate', description='Destination Plate')

# max transfer vol
fillField = Text(value='Max Volume per Well', description='Transfer Vol')


# row and col selectors
colBound = BoundedIntText(
    value=2,
    min=2,
    max=22,
    step=1,
    description='Column Index'
)

rowBound = BoundedIntText(
    value=2,
    min=0,
    max=16,
    step=1,
    description='Row Index'
)

# go button
submit = Button(
    description='Add Compound',
    disabled=False,
    button_style='',
    tooltip='Only press when ready',
    icon=''
)

compounds = []

# bind button to handler
submit.on_click(button_handler)

# show me the bits
display(nameDrop)
display(diluteDrop)
display(destField)
display(fillField)
display(colBound)
display(rowBound)

display(submit)

Dropdown(description='Compound', options=('ABT-737', 'Linifanib (ABT-869)', 'Veliparib (ABT-888)', 'Axitinib',…

Dropdown(description='Select Dilute amount', options=('log2', '1/2'), value=None)

Text(value='Destination Plate', description='Destination Plate')

Text(value='Max Volume per Well', description='Transfer Vol')

BoundedIntText(value=2, description='Column Index', max=22, min=2)

BoundedIntText(value=2, description='Row Index', max=16)

Button(description='Add Compound', style=ButtonStyle(), tooltip='Only press when ready')

In [23]:
type(compounds[0])

pandas.core.frame.DataFrame

In [ ]:
well_rows = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P'] # non-control rows
well_cols = [str(x) for x in range(1, 25)] # non-contorl columns

def make_control(destID):
    """
    creates controls and stores them in a dataframe
    assumes that control schema is universal
    args:
        destID - The Plate ID
    returns:
        df - dataframe of positvie and negative controls 
    """
    neg_row, pos_row = [],[]
    for y in well_rows:
        for x in ['1','2']:
            neg_row.append(y+x)
        for j in ['23','24']:
            pos_row.append(y+j)
            
    neg = pd.DataFrame.from_dict({
        'sourceID':['NC' for i in range(len(neg_row))],
        'sourceWell':['NC' for i in range(len(neg_row))],
        'destID':[destID for i in range(len(neg_row))], # destination plate ID
        'destWell':neg_row,
        'transferVol':[0.0 for i in range(len(neg_row))],
        'backfillVol':[0.0 for i in range(len(neg_row))],
        'compound':['NC' for i in range(len(neg_row))]
    })
    
    pos = pd.DataFrame({
        'sourceID':['PC' for i in range(len(pos_row))],
        'sourceWell':['PC' for i in range(len(pos_row))],
        'destID':[destID for i in range(len(pos_row))], # destination plate ID
        'destWell':pos_row,
        'transferVol':[0.0 for i in range(len(pos_row))],
        'backfillVol':[0.0 for i in range(len(pos_row))],
        'compound':['PC' for i in range(len(pos_row))]
    })
    
    df = pd.concat([neg, pos])
    
    return df

In [ ]:
df = make_control('someplate')

In [ ]:
plate = pd.concat([df,dt])

In [ ]:
plate.head()

In [ ]:
piv = plate
piv[['well', 'wellNum']] = plate['destWell'].str.extract('([A-P])([0-9]{1,2})',expand=True)
table = pd.pivot_table(piv, values='transferVol', index='well', columns='wellNum')

In [4]:
in_text = Text()
out_text = Text()

def bind_input(sender):
    out_text.value = in_text.value

in_text.on_submit(bind_input)

display(in_text)

display(out_text)

Text(value='')

Text(value='')

In [8]:

submit

Button(description='Add Compound', style=ButtonStyle(), tooltip='Only press when ready')